In [2]:
#!pip install altair pandas numpy
import altair as alt
import pandas as pd
import numpy as np
pd.options.display.max_columns = None

# norms
# mean sj: (1,1,0,0,0,0,1,1)
# nice sj: (1,1,0,1,0,1,1,1)

ebnorms_b = ["(0,0,1,1,0,0,1,1)", "(0,0,0,1,0,1,1,1)", "(0,0,1,1,0,1,1,1)"]

ebnorms_n = ["IS normal", "Emotion-Strict IS", "Emotion-Lenient IS"]

norms_simple_b = ["(1,0,0,1)", "(1,0,1,1)", "(0,0,0,1)", "(0,0,1,1)"]

norms_simple_n = ["SJ", "SS", "SH", "IS"]

columns = ["EBNorm", "Norm", "z", "g", "mu", "chi", "eps", "alpha", "gamma", "pdx", "b", "c", "beta", 
           "acr", "B", "G", "AllD", "Disc", "pDisc", "AllC", "m", "n", "ET"]

#columns = ["EBNorm", "Norm", "z", "g", "mu", "chi", "eps", "alpha", "gamma", "pdx", 
#           "acr", "B", "G", "AllD", "Disc", "pDisc", "AllC", "m", "n", "ET"]
dtype={'ET': str}

path = "outputs/results_bc.txt"
#path = "outputs/results.txt"

def load_and_process() -> pd.DataFrame: 
    
    data: pd.DataFrame = pd.read_csv(path, sep="\t", header=None, 
                                     names=columns, dtype=dtype, index_col=False)
    data.eps = data.eps / data.z
    data.chi = data.chi / data.z
    data.alpha = data.alpha / data.z
    data.dropna()
    data.EBNorm = data.EBNorm.replace(ebnorms_b, ebnorms_n)
    data.Norm = data.Norm.replace(norms_simple_b, norms_simple_n)
    return data

def filter(frame: pd.DataFrame, feature: str, condition) -> pd.DataFrame:
    return frame[frame[feature]==condition]
    
data = load_and_process()


In [3]:
data[data.EBNorm!="Emotion-Lenient IS"]

,EBNorm,Norm,z,g,mu,chi,eps,alpha,gamma,pdx,b,c,beta,acr,B,G,AllD,Disc,pDisc,AllC,m,n,ET
1620,IS normal,IS,50.0,5000.0,0.1,0.002,0.002,0.002,0.00,True,5.0,1.0,1.0,6.535,1.00,0.00,1.0,0.00,0.0,0.00,1.00,0.00,000
1624,IS normal,IS,50.0,5000.0,0.1,0.002,0.002,0.002,0.00,True,5.0,1.0,1.0,35.341,1.00,0.00,1.0,0.00,0.0,0.00,0.60,0.40,000
1628,IS normal,IS,50.0,5000.0,0.1,0.002,0.002,0.002,0.00,True,5.0,1.0,1.0,33.795,0.02,0.98,0.0,0.86,0.0,0.14,0.14,0.86,101
1630,IS normal,IS,50.0,5000.0,0.1,0.002,0.002,0.002,0.00,True,5.0,1.0,1.0,18.201,0.98,0.02,1.0,0.00,0.0,0.00,0.00,1.00,100
1631,IS normal,IS,50.0,5000.0,0.1,0.002,0.002,0.002,0.00,True,5.0,1.0,1.0,8.213,1.00,0.00,1.0,0.00,0.0,0.00,1.00,0.00,000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7787,IS normal,IS,50.0,5000.0,0.1,0.002,0.002,0.002,0.75,True,5.0,1.0,1.0,5.756,1.00,0.00,1.0,0.00,0.0,0.00,0.00,1.00,100
7788,IS normal,IS,50.0,5000.0,0.1,0.002,0.002,0.002,0.85,True,5.0,1.0,1.0,15.563,1.00,0.00,1.0,0.00,0.0,0.00,0.00,1.00,100
7789,IS normal,IS,50.0,5000.0,0.1,0.002,0.002,0.002,0.85,True,5.0,1.0,1.0,7.884,1.00,0.00,1.0,0.00,0.0,0.00,1.00,0.00,000
7790,IS normal,IS,50.0,5000.0,0.1,0.002,0.002,0.002,0.95,True,5.0,1.0,1.0,18.122,1.00,0.00,1.0,0.00,0.0,0.00,0.00,1.00,100


In [10]:
data = load_and_process()

b = 5
value = 0.002
filtered = filter(data, "chi", value)
filtered = filter(filtered, "alpha", value)
filtered = filter(filtered, "eps", value)
filtered = filter(filtered, "b", b)
filtered = filter(filtered, "beta", 1)
#filtered = filter(filtered, "EBNorm", "Emotion-Lenient Image Scoring")


new_cols = ["gamma", "Emotion-Based Image Scoring", "EBIS std", "Image Scoring", "IS std"]
#, "MostCommonEP"
#new_cols = ["gamma", "Emotion-Based Image Scoring", "std", "Image Scoring"]
#new_cols = ["gamma", "Emotion-Strict Image Scoring", "Emotion-Lenient Image Scoring", "Image Scoring"]
new_df = pd.DataFrame(columns=new_cols)
for gamma in filtered.gamma.unique():
    subset = filtered[filtered.gamma==gamma]
    n = len(subset)
    #print(subset)
    #diff = subset[subset.EBNorm=="IS Empirical"].acr.mean() - subset[subset.EBNorm=="IS normal"].acr.mean()
    entry = [gamma, 
             #subset[subset.EBNorm=="Emotion-Strict IS"].acr.mean(), 
             subset[subset.EBNorm=="Emotion-Lenient IS"].acr.mean(), 
             subset[subset.EBNorm=="Emotion-Lenient IS"].acr.std(), 
             subset[subset.EBNorm=="IS normal"].acr.mean(),
             subset[subset.EBNorm=="IS normal"].acr.std(),
             #subset.shape[0]
             #subset[subset.EBNorm=="Emotion-Lenient IS"].ET.mode()
            ]
    new_df.loc[len(new_df)] = entry
    
new_df

,gamma,Emotion-Based Image Scoring,EBIS std,Image Scoring,IS std
0,0.00,14.921160,9.705640,15.048760,8.547789
1,0.10,24.047000,13.299010,14.531490,9.328750
2,0.20,37.786900,24.691476,13.974673,8.329595
3,0.30,44.710612,26.611827,13.483920,7.451260
4,0.40,44.273420,26.500991,14.525551,8.198878
5,0.50,50.453660,28.225745,14.415120,7.039898
6,0.60,57.572780,23.660247,14.430204,8.079194
7,0.70,59.025200,25.020812,12.599740,7.209739
8,0.80,65.586960,20.505997,16.913360,7.925396
9,0.90,74.000060,17.459990,12.505540,7.903517


In [14]:
points_ebis = alt.Chart(new_df).mark_point(filled=True, size=80).encode(
    x='gamma',
    y='Emotion-Based Image Scoring',
    #color='Social Norm:N',
    #shape='Social Norm:N'
)

error_bars_ebis = alt.Chart(new_df).mark_errorbar(extent='ci', ticks=True, opacity=0.5).encode(
    x='gamma',
    y=alt.Y('Emotion-Based Image Scoring'),
    yError='EBIS std'
)

points_is = alt.Chart(new_df).mark_point(filled=True, size=80, color='red', shape="square").encode(
    x='gamma',
    y='Image Scoring',
    #shape='Social Norm:N'
)

error_bars_is = alt.Chart(new_df).mark_errorbar(extent='ci', ticks=True, color='orange', opacity=0.5).encode(
    x='gamma',
    y=alt.Y('Image Scoring'),
    yError='IS std'
)

# Regression line for Emotion-Based Image Scoring
regression_ebis = alt.Chart(new_df).transform_regression(
    'gamma', 'Emotion-Based Image Scoring'
).mark_line(color='lightblue', opacity=0.3).encode(
    x='gamma',
    y='Emotion-Based Image Scoring'
)

# Regression line for Image Scoring
regression_is = alt.Chart(new_df).transform_regression(
    'gamma', 'Image Scoring'
).mark_line(color='red', opacity=0.3).encode(
    x='gamma',
    y='Image Scoring'
)

# Combine all layers
chart = (
    regression_ebis + regression_is + error_bars_is + error_bars_ebis + points_is + points_ebis
).properties(
    #title="b/c = " + str(b),  # Replace with your desired title
    width=450
).configure_title(
    fontSize=16,  # Adjust font size for the title
    anchor='middle'  # Center the title
).encode(
    x=alt.X('gamma', title='Probability of accounting for emotion'),
    y=alt.Y(
        'Emotion-Based Image Scoring',
        title='Cooperation Index',
        scale=alt.Scale(domain=[0, 100])  # Set y-axis range from 0 to 100
    )
)

chart

alt.LayerChart(...)

In [9]:
data = load_and_process()

value = 0.002
beta = 0.6
filtered = filter(data, "chi", value)
filtered = filter(filtered, "alpha", value)
filtered = filter(filtered, "eps", value)
filtered = filter(filtered, "b", 5)
filtered = filter(filtered, "beta", beta)


new_cols = ["gamma", "Emotion-Based Image Scoring", "EBIS std", "Image Scoring", "IS std"]
#, "MostCommonEP"
#new_cols = ["gamma", "Emotion-Based Image Scoring", "std", "Image Scoring"]
#new_cols = ["gamma", "Emotion-Strict Image Scoring", "Emotion-Lenient Image Scoring", "Image Scoring"]
new_df = pd.DataFrame(columns=new_cols)
for gamma in filtered.gamma.unique():
    subset = filtered[filtered.gamma==gamma]
    #print(subset)
    #diff = subset[subset.EBNorm=="IS Empirical"].acr.mean() - subset[subset.EBNorm=="IS normal"].acr.mean()
    entry = [gamma, 
             #subset[subset.EBNorm=="Emotion-Strict IS"].acr.mean(), 
             subset[subset.EBNorm=="Emotion-Lenient IS"].acr.mean(), 
             subset[subset.EBNorm=="Emotion-Lenient IS"].acr.std(), 
             subset[subset.EBNorm=="IS normal"].acr.mean(),
             subset[subset.EBNorm=="IS normal"].acr.std(),
             #subset.shape[0]
             #subset[subset.EBNorm=="Emotion-Lenient IS"].ET.mode()
            ]
    new_df.loc[len(new_df)] = entry
    
new_df

,gamma,Emotion-Based Image Scoring,EBIS std,Image Scoring,IS std
0,0.0,18.21950,9.196420,14.922960,8.503369
1,0.2,27.80838,21.455641,18.792700,9.390725
2,0.4,45.83508,24.495158,18.350340,9.390008
3,0.6,53.66532,23.929287,16.406300,8.946227
4,0.8,65.14800,18.551615,15.471265,9.972225
5,1.0,9.46136,7.670825,14.325140,9.010817


In [26]:
points_ebis = alt.Chart(new_df).mark_point(filled=True, size=70).encode(
    x='gamma',
    y='Emotion-Based Image Scoring',
    #color='Social Norm:N',
    #shape='Social Norm:N'
)

error_bars_ebis = alt.Chart(new_df).mark_errorbar(extent='ci', ticks=True, opacity=0.8).encode(
    x='gamma',
    y=alt.Y('Emotion-Based Image Scoring'),
    yError='EBIS std'
)

points_is = alt.Chart(new_df).mark_point(filled=True, size=70, color='red').encode(
    x='gamma',
    y='Image Scoring',
    #shape='Social Norm:N'
)

error_bars_is = alt.Chart(new_df).mark_errorbar(extent='ci', ticks=True, color='orange', opacity=0.8).encode(
    x='gamma',
    y=alt.Y('Image Scoring'),
    yError='IS std'
)

# Combine all layers
chart = (
    points_is + error_bars_is + points_ebis + error_bars_ebis
).properties(
    title="beta = " + str(beta)  # Replace with your desired title
).configure_title(
    fontSize=16,  # Adjust font size for the title
    anchor='middle'  # Center the title
).encode(
    x=alt.X('gamma', title='Probability of accounting for emotion'),
    y=alt.Y(
        'Emotion-Based Image Scoring',
        title='Cooperation Index',
        scale=alt.Scale(domain=[0, 100])  # Set y-axis range from 0 to 100
    )
)

chart

alt.LayerChart(...)

In [110]:
data = load_and_process()

ebnorm = "Emotion-Lenient IS"

value = 0.002
filtered = filter(data, "chi", value)
filtered = filter(filtered, "alpha", value)
filtered = filter(filtered, "eps", value)
filtered = filter(filtered, "EBNorm", ebnorm)
filtered = filter(filtered, "b", 5)
filtered = filter(filtered, "beta", 1)

eps_df = pd.DataFrame(columns=["gamma", "Cooperative EP", "Competitive EP"])
strats_df = pd.DataFrame(columns=["gamma", "AllD", "pDisc", "Disc", "AllC"])
reps_df = pd.DataFrame(columns=["gamma", "good", "bad"])

for gamma in filtered.gamma.unique():
    subset = filtered[filtered.gamma==gamma]
    # CHANGE "MEAN" TO SOMETHING ELSE IN THE DATASET
    
    entry_eps = [gamma, subset.n.mean(), subset.m.mean()]
    entry_strats = [gamma, subset.AllD.mean(), subset.pDisc.mean(), subset.Disc.mean(), subset.AllC.mean()]
    entry_reps = [gamma, subset.G.mean(), subset.B.mean()]
    
    eps_df.loc[len(eps_df)] = entry_eps
    strats_df.loc[len(strats_df)] = entry_strats
    reps_df.loc[len(reps_df)] = entry_reps
    
melted_eps = eps_df.melt('gamma', var_name='emotion profile', value_name='frequency')
melted_strats = strats_df.melt('gamma', var_name='strategy', value_name='frequency')
melted_reps = reps_df.melt('gamma', var_name='reputation', value_name='frequency')

In [111]:
# Remove what I don't want to plot
#melted_strats = melted_strats.drop(melted_strats[melted_strats.strat=="AllC"].index)
#melted_strats = melted_strats.drop(melted_strats[melted_strats.strat=="pDisc"].index)
melted_reps = melted_reps.drop(melted_reps[melted_reps.reputation=="bad"].index)
melted_eps = melted_eps.drop(melted_eps[melted_eps["emotion profile"]=="Competitive EP"].index)
melted_strats

,gamma,strategy,frequency
0,0.0,AllD,0.783200
1,0.1,AllD,0.614000
2,0.2,AllD,0.586400
3,0.3,AllD,0.502857
4,0.4,AllD,0.616800
5,0.5,AllD,0.478800
6,0.6,AllD,0.410000
7,0.7,AllD,0.335600
8,0.8,AllD,0.238800
9,0.9,AllD,0.298800


In [116]:
strat_chart = alt.Chart(melted_strats).mark_line(
    interpolate="monotone"
    ).encode(
        x='gamma',
        y='frequency',
        color=alt.Color('strategy:N', scale=alt.Scale(scheme="set2"), sort='ascending')
)


reps_chart = alt.Chart(melted_reps).mark_line(
    #size=50,
    interpolate="monotone",
    strokeDash=[3,3]
).encode(
    x='gamma',
    y='frequency',
    color=alt.Color('reputation', sort='ascending')
)

eps_chart = alt.Chart(melted_eps).mark_point(
    size=90,
    filled=True
).encode(
    x='gamma',
    y='frequency',
    #shape='emotional profile:N',
    color=alt.Color('emotional profile:N', scale=alt.Scale(scheme="set2"), sort='descending')
)

In [122]:
(strat_chart + eps_chart + reps_chart).resolve_scale(color='independent', shape='independent').properties(
    #title="b/c = " + str(b),  # Replace with your desired title
    width=380)

alt.LayerChart(...)

In [118]:
strat_chart + reps_chart

alt.LayerChart(...)

In [20]:
def param_heatmap(chosen_parameter: str, feature: str, norm: str):
    data = load_and_process()

    # Initial filtering
    filtered = filter(data, "z", 50)
    filtered = filter(filtered, "eps", 0.00002) if chosen_parameter != "eps" else filtered
    filtered = filter(filtered, "chi", 0.00002) if chosen_parameter != "chi" else filtered
    filtered = filter(filtered, "alpha", 0.00002) if chosen_parameter != "alpha" else filtered
    
    filtered = filter(filtered, "EBNorm", norm)

    # Define columns based on the chosen parameter
    new_cols = ["gamma", chosen_parameter, "value"]
    new_df = pd.DataFrame(columns=new_cols)

    #gammas = [round(a, 2) for a in np.arange(0.9, 1.01, 0.01)]
    gammas = filtered.gamma.unique()

    # Define values based on the chosen parameter
    parameter_values = [0.00002, 0.0002, 0.002, 0.02, 0.2]

    for gamma in gammas:
        for parameter_value in parameter_values:
            super_filtered = filtered[(filtered.gamma == gamma) & (filtered[chosen_parameter] == parameter_value)]

            feature_mean: float = super_filtered[feature].mean()

            new_df.loc[len(new_df)] = [gamma, parameter_value, feature_mean]

    # Define feature for Altair chart
    view_feature = "value:Q"

    # Create Altair chart
    heatmap = alt.Chart(new_df).mark_rect().encode(
        x=f"{chosen_parameter}:O",
        y=alt.Y('gamma:O', sort=alt.EncodingSortField('gamma', order='descending')),
        color=alt.Color(view_feature, scale=alt.Scale(domain=[0, 1], scheme='blueorange'), sort="ascending")
    ).properties(
        height=400,
        width=400,
        title=chosen_parameter + " x " + feature + " x " + norm
    )

    # Add text annotations to the chart
    text = heatmap.mark_text().encode(
        text=alt.Text(view_feature, format=".2f"),
        color=alt.value('black')
    )

    # Display the chart with text annotations
    #heatmap + text
    return new_df, (heatmap + text)

In [21]:
d, c = param_heatmap("alpha", "AllD", "IS Empirical")
c

alt.LayerChart(...)